<a href="https://colab.research.google.com/github/Satyajit24behera/Py-feat-webapp/blob/main/py_feat_streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 0: Prerequisite

Get your Authentication Tokens from Ngrok and save it in a notebook. You need it later to run app.

1.Go to https://dashboard.ngrok.com/get-started/setup signup for free


2.Once you are logged in go to https://dashboard.ngrok.com/get-started/your-authtoken and copy+paste token and save it for later use.


**NOTE**: If you already have auth token the you can skip above steps and use that token


In [56]:
!git clone https://github.com/cosanlab/py-feat


Cloning into 'py-feat'...
remote: Enumerating objects: 8947, done.
remote: Counting objects: 100% (8947/8947), done.
remote: Compressing objects: 100% (3033/3033), done.
remote: Total 8947 (delta 5439), reused 8886 (delta 5392), pack-reused 0
Receiving objects: 100% (8947/8947), 180.21 MiB | 24.36 MiB/s, done.
Resolving deltas: 100% (5439/5439), done.


In [57]:
cd py-feat

/content/py-feat/webapp/webapp/py-feat


In [58]:
!git pull origin au_fixes

From https://github.com/cosanlab/py-feat
 * branch            au_fixes   -> FETCH_HEAD
Updating 4beba11..b1021db
Fast-forward
 docs/extra_tutorials/08_train_hogs.ipynb           | 2440 ++++++++++++++------
 feat/au_detectors/StatLearning/SL_test.py          |  101 +-
 feat/detector.py                                   |    2 +-
 feat/emo_detectors/StatLearning/EmoSL_test.py      |   10 +-
 feat/facepose_detectors/img2pose/img2pose_model.py |    2 +-
 feat/pretrained.py                                 |   16 +-
 feat/resources/model_list.json                     |   57 +-
 7 files changed, 1922 insertions(+), 706 deletions(-)


In [59]:
pip install -r requirements-dev.txt

In [60]:
pip install setup.py

ERROR: Could not find a version that satisfies the requirement setup.py (from versions: none)
ERROR: No matching distribution found for setup.py


In [61]:
pip install  py-feat

In [62]:
pip install matplotlib

# Step 1: Setup Chunk
 Step 1.1: Install dependencies

In [63]:
!pip install streamlit


In [64]:
pip install pyngrok


Step 1.2: Replace XXXXXX with your ngrok token

In [65]:
#Replace the xxxx with authtoken
!ngrok authtoken 2RdaFL2BBdIi9sVDUb2sJX2Kzbq_83JxMGDVAjNACaFVUL6ks

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


Step 1.3 Git hub Cloning

In [66]:
!git clone https://github.com/Satyajit24behera/Py-feat-webapp  webapp

Cloning into 'webapp'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 36 (delta 15), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (36/36), 19.97 KiB | 1.43 MiB/s, done.


#Step 2: Hosting the Web app

In [67]:
from pyngrok import ngrok
public_url = ngrok.connect(port='8501')
print (public_url)
!streamlit run --server.port 8501 https://raw.githubusercontent.com/Satyajit24behera/Py-feat-webapp/main/py_feat_streamlit.py >/dev/null

http://d7d3-35-221-154-82.ngrok-free.app


#### If you want to stop the app then stop execution of above cell and then run below code

In [68]:
!pgrep streamlit
ngrok.kill()